## Домашнее задание по теме «Python для анализа данных: pandas»

In [176]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [69]:
df_k = pd.read_csv('/Users/aleksandr/Downloads/keywords.csv')
df_r = pd.read_csv('/Users/aleksandr/Downloads/ratings.csv')
df_m = pd.read_csv('/Users/aleksandr/Downloads/movies.csv')

### **Задание 1**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'. Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

    geo_data = {
        'Центр': ['москва', 'тула', 'ярославль'],
        'Северо-Запад': ['петербург', 'псков', 'мурманск'],
        'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
    }

Результат классификации запишите в отдельный столбец region.

### Решение

In [7]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [8]:
def find(parse):
    for k, v in geo_data.items():
        if parse.lower() in v:
            return k
    return 'null'

In [9]:
df_k['region'] = df_k['keyword'].apply(find)
df_k[df_k['region'] != 'null']

,keyword,shows,region
1259,москва,153050,Центр
5858,владивосток,43038,Дальний Восток
6075,ярославль,41720,Центр
8977,мурманск,29984,Северо-Запад
9084,хабаровск,29655,Дальний Восток
11222,псков,24885,Северо-Запад
11785,тула,23898,Центр
20823,петербург,14859,Северо-Запад
21445,сахалин,14494,Дальний Восток


### **Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

### Решение

In [17]:
def class_rating(rating):
    if rating <= 2:
        return 'низкий рейтинг'
    elif rating <= 4:
        return 'средний рейтинг'
    else:
        return 'высокий рейтинг'

In [23]:
mean_r = df_r.groupby('movieId').mean()[['rating']]
mean_r['class'] = mean_r['rating'].apply(class_rating)
res = df_m.join(mean_r, how='left', on='movieId').set_index('movieId')

In [24]:
res.head()

,title,genres,rating,class
movieId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.921240,средний рейтинг
2,Jumanji (1995),Adventure|Children|Fantasy,3.211977,средний рейтинг
3,Grumpier Old Men (1995),Comedy|Romance,3.151040,средний рейтинг
4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.861393,средний рейтинг
5,Father of the Bride Part II (1995),Comedy,3.064592,средний рейтинг


### **Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. 

Ответ дайте в днях.

### Решение

In [140]:
ratings = pd.read_csv('/Users/aleksandr/Downloads/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [141]:
ratings['timestamp'] = ratings['timestamp'].apply(datetime.utcfromtimestamp)
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [142]:
grouped = ratings.groupby('userId').count().reset_index()
grouped.head()

,userId,movieId,rating,timestamp
0,1,175,175,175
1,2,61,61,61
2,3,187,187,187
3,4,28,28,28
4,5,66,66,66


In [147]:
ratings_filtered = grouped[grouped['timestamp'] >= 100]
ratings_filtered.head()

,userId,movieId,rating,timestamp
0,1,175,175,175
2,3,187,187,187
6,7,276,276,276
10,11,504,504,504
13,14,243,243,243


In [148]:
ratings_filtered['userId'].tolist()

[1,
 3,
 7,
 11,
 14,
 18,
 21,
 22,
 23,
 24,
 25,
 29,
 31,
 35,
 46,
 51,
 53,
 54,
 56,
 58,
 61,
 69,
 70,
 73,
 77,
 82,
 88,
 89,
 90,
 91,
 96,
 101,
 103,
 104,
 109,
 112,
 116,
 124,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 137,
 140,
 144,
 147,
 148,
 152,
 153,
 154,
 156,
 157,
 158,
 160,
 162,
 165,
 168,
 169,
 170,
 184,
 189,
 201,
 205,
 206,
 208,
 215,
 218,
 220,
 229,
 232,
 237,
 238,
 239,
 241,
 245,
 247,
 248,
 251,
 254,
 258,
 264,
 265,
 266,
 267,
 269,
 271,
 273,
 278,
 279,
 280,
 283,
 284,
 285,
 294,
 295,
 298,
 304,
 306,
 309,
 311,
 312,
 313,
 314,
 316,
 317,
 318,
 330,
 331,
 334,
 335,
 337,
 340,
 343,
 344,
 346,
 347,
 348,
 349,
 351,
 358,
 359,
 360,
 367,
 370,
 375,
 377,
 378,
 380,
 383,
 386,
 387,
 388,
 389,
 394,
 395,
 399,
 401,
 402,
 407,
 409,
 411,
 412,
 413,
 415,
 417,
 421,
 422,
 424,
 425,
 427,
 430,
 435,
 436,
 440,
 445,
 448,
 449,
 451,
 455,
 456,
 457,
 459,
 461,
 462,
 465,
 466,
 469,
 471,
 473,
 4

In [161]:
ratings_df = ratings_filtered[ratings_filtered['userId'].isin(ratings_filtered['userId'].tolist())]
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,175,175,175
2,3,187,187,187
6,7,276,276,276
10,11,504,504,504
13,14,243,243,243


In [297]:
# ч.2
day_r = pd.read_csv('/Users/aleksandr/Downloads/ratings.csv')
day_r.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [298]:
day_r['timestamp'] = day_r['timestamp'].apply(datetime.utcfromtimestamp)
day_r.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [299]:
# day_r.columns

In [300]:
day_r2 = day_r.drop(['movieId', 'rating'], axis=1)

In [301]:
day_r2.head()

,userId,timestamp
0,1,2005-04-02 23:53:47
1,1,2005-04-02 23:31:16
2,1,2005-04-02 23:33:39
3,1,2005-04-02 23:32:07
4,1,2005-04-02 23:29:40


In [309]:
dd = day_r2.groupby('userId')['timestamp'].apply(lambda x: x.max() - x.min())
dd.head()

userId
1   204 days 20:50:03
2     0 days 00:06:56
3     3 days 05:29:51
4     0 days 00:13:49
5     1 days 01:13:13
Name: timestamp, dtype: timedelta64[ns]

In [349]:
merge_df = ratings_df.merge(dd, on='userId', how='inner')
merge_df.head()

,userId,movieId,rating,timestamp_x,timestamp_y
0,1,175,175,175,204 days 20:50:03
1,3,187,187,187,3 days 05:29:51
2,7,276,276,276,0 days 01:21:02
3,11,504,504,504,740 days 21:34:32
4,14,243,243,243,0 days 03:18:01


In [350]:
del merge_df['timestamp_x']

print(merge_df.info())
print('---\n')
print(merge_df.columns)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52596 entries, 0 to 52595
Data columns (total 4 columns):
userId         52596 non-null int64
movieId        52596 non-null int64
rating         52596 non-null int64
timestamp_y    52596 non-null timedelta64[ns]
dtypes: int64(3), timedelta64[ns](1)
memory usage: 2.0 MB
None
---

Index(['userId', 'movieId', 'rating', 'timestamp_y'], dtype='object')


In [347]:
merge_df = merge_df.rename(columns={'timestamp_y':'lifetime'}).sort_values('lifetime', ascending=False)
merge_df.head()

,userId,movieId,rating,lifetime
30248,79366,2034,2034,6405 days 06:30:21
22713,59414,2780,2780,6391 days 15:03:19
34894,91435,742,742,6389 days 02:01:22
14104,36869,764,764,6388 days 23:43:47
15118,39579,1024,1024,6370 days 12:04:20


In [ ]:
'''pass'''

In [258]:
# lifetime_day_r2 = day_r2.groupby('userId')[['timestamp']].agg(['max', 'min']).reset_index()
# lifetime_day_r2.head(10)
# # type(lifetime_day_r2)

userId           timestamp                    
                         max                 min
0      1 2005-04-02 23:56:41 2004-09-10 03:06:38
1      2 2000-11-21 15:36:54 2000-11-21 15:29:58
2      3 1999-12-14 12:54:59 1999-12-11 07:25:08
3      4 1996-08-24 09:40:54 1996-08-24 09:27:05
4      5 1996-12-26 16:28:48 1996-12-25 15:15:35
5      6 1997-03-13 17:58:37 1997-03-13 17:50:52
6      7 2002-01-16 19:29:08 2002-01-16 18:08:06
7      8 1996-06-18 08:46:43 1996-06-05 11:11:21
8      9 2001-07-01 20:53:54 2001-07-01 20:25:33
9     10 1999-11-25 02:46:26 1999-11-25 02:32:02

In [261]:
# joined = ratings_df.merge(lifetime_day_r2, on='userId', how='inner')

/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [277]:
# joined2 = joined.drop('timestamp', axis=1)
# joined2.head()

,userId,movieId,rating,"(timestamp, max)","(timestamp, min)"
0,1,175,175,2005-04-02 23:56:41,2004-09-10 03:06:38
1,3,187,187,1999-12-14 12:54:59,1999-12-11 07:25:08
2,7,276,276,2002-01-16 19:29:08,2002-01-16 18:08:06
3,11,504,504,2011-01-12 01:36:41,2009-01-01 04:02:09
4,14,243,243,2008-10-29 22:50:27,2008-10-29 19:32:26


In [296]:
# joined2.columns

Index(['userId', 'movieId', 'rating', 'max', 'min'], dtype='object')

In [292]:
# joined2.columns = ['userId', 'movieId', 'rating', 'max', 'min']
# joined2.head()

,userId,movieId,rating,max,min
0,1,175,175,2005-04-02 23:56:41,2004-09-10 03:06:38
1,3,187,187,1999-12-14 12:54:59,1999-12-11 07:25:08
2,7,276,276,2002-01-16 19:29:08,2002-01-16 18:08:06
3,11,504,504,2011-01-12 01:36:41,2009-01-01 04:02:09
4,14,243,243,2008-10-29 22:50:27,2008-10-29 19:32:26


In [311]:
# joined.info()
# joined['new'] = joined.apply(joined[('timestamp', 'max')] - joined[('timestamp', 'min')], axis=1)

In [ ]:
'''pass'''

### **Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

### Решение

In [48]:
years = list(map(str, range(1950, 2011)))

In [49]:
mean_r = df_r.groupby('movieId').mean()[['rating']]
res = df_m.join(mean_r, how='left', on='movieId').set_index('movieId')

In [50]:
res['year'] = res['title'].str.extract(r'(\d{4})')
res['year'] = res['year'].apply(lambda year: int(year) if year in years else 1900)

In [51]:
df_raiting = res.groupby('year')[['rating']].mean().sort_values(by='rating', ascending=False)

In [52]:
df_raiting.head()

,rating
year,
1950,3.316406
1961,3.315166
1962,3.305996
1953,3.302317
1964,3.300282


In [ ]:
pass